# HiveJoin

### 不同类型的联接：

* JOIN
* LEFT OUTER JOIN
* RIGHT OUTER JOIN
* FULL OUTER JOIN

![title](img/join.png)

## 示例:


| ID | NAME     | AGE | ADDRESS   | SALARY | 
|----|----------|----- |-----------|----------| 
| 1  | Ramesh   | 32  | Ahmedabad | 2000.00  |  
| 2  | Khilan   | 25  | Delhi     | 1500.00  |  
| 3  | kaushik  | 23  | Kota      | 2000.00  | 
| 4  | Chaitali | 25  | Mumbai    | 6500.00  | 
| 5  | Hardik   | 27  | Bhopal    | 8500.00  | 
| 6  | Komal    | 22  | MP        | 4500.00  | 
| 7  | Muffy    | 24  | Indore    | 10000.00 | 



|OID  | DATE                | CUSTOMER_ID | AMOUNT | 
|-----|---------------------|-------------|--------|
| 102 | 2009-10-08 00:00:00 |           3 | 3000   | 
| 100 | 2009-10-08 00:00:00 |           3 | 1500   | 
| 101 | 2009-11-20 00:00:00 |           2 | 1560   | 
| 103 | 2008-05-20 00:00:00 |           4 | 2060   | 


## JOIN

In [ ]:
hive> SELECT c.ID, c.NAME, c.AGE, o.AMOUNT 
   > FROM CUSTOMERS c JOIN ORDERS o 
   > ON (c.ID = o.CUSTOMER_ID);

#### 成功执行查询后，能看到以下回应：

| ID | NAME     | AGE | AMOUNT | 
|-----|---------------------|-------------|--------|
| 3  | kaushik  | 23  | 3000   | 
| 3  | kaushik  | 23  | 1500   | 
| 2  | Khilan   | 25  | 1560   | 
| 4  | Chaitali | 25  | 2060   | 

## LEFT OUTER JOIN

In [ ]:
hive> SELECT c.ID, c.NAME, o.AMOUNT, o.DATE 
   > FROM CUSTOMERS c 
   > LEFT OUTER JOIN ORDERS o 
   > ON (c.ID = o.CUSTOMER_ID);

#### 成功执行查询后，能看到以下回应：

| ID | NAME     | AMOUNT | DATE                | 
|-----|---------------------|-------------|--------|
| 1  | Ramesh   | NULL   | NULL                | 
| 2  | Khilan   | 1560   | 2009-11-20 00:00:00 | 
| 3  | kaushik  | 3000   | 2009-10-08 00:00:00 | 
| 3  | kaushik  | 1500   | 2009-10-08 00:00:00 | 
| 4  | Chaitali | 2060   | 2008-05-20 00:00:00 | 
| 5  | Hardik   | NULL   | NULL                | 
| 6  | Komal    | NULL   | NULL                | 
| 7  | Muffy    | NULL   | NULL                | 

## RIGHT OUTER JOIN

In [ ]:
hive> SELECT c.ID, c.NAME, o.AMOUNT, o.DATE 
   > FROM CUSTOMERS c 
   > RIGHT OUTER JOIN ORDERS o 
   > ON (c.ID = o.CUSTOMER_ID);

#### 成功执行查询后，能看到以下回应：

| ID   | NAME     | AMOUNT | DATE                | 
|-----|---------------------|-------------|--------|
| 3    | kaushik  | 3000   | 2009-10-08 00:00:00 | 
| 3    | kaushik  | 1500   | 2009-10-08 00:00:00 | 
| 2    | Khilan   | 1560   | 2009-11-20 00:00:00 | 
| 4    | Chaitali | 2060   | 2008-05-20 00:00:00 | 


## FULL OUTER JOIN

In [ ]:
hive> SELECT c.ID, c.NAME, o.AMOUNT, o.DATE 
   > FROM CUSTOMERS c 
   > FULL OUTER JOIN ORDERS o 
   > ON (c.ID = o.CUSTOMER_ID);

#### 成功执行查询后，能看到以下回应：

| ID   | NAME     | AMOUNT | DATE                | 
|-----|---------------------|-------------|--------|
| 1    | Ramesh   | NULL   | NULL                | 
| 2    | Khilan   | 1560   | 2009-11-20 00:00:00 | 
| 3    | kaushik  | 3000   | 2009-10-08 00:00:00 | 
| 3    | kaushik  | 1500   | 2009-10-08 00:00:00 | 
| 4    | Chaitali | 2060   | 2008-05-20 00:00:00 | 
| 5    | Hardik   | NULL   | NULL                | 
| 6    | Komal    | NULL   | NULL                |
| 7    | Muffy    | NULL   | NULL                |  
| 3    | kaushik  | 3000   | 2009-10-08 00:00:00 | 
| 3    | kaushik  | 1500   | 2009-10-08 00:00:00 | 
| 2    | Khilan   | 1560   | 2009-11-20 00:00:00 | 
| 4    | Chaitali | 2060   | 2008-05-20 00:00:00 | 

## 写 join 查询时，需要注意几个关键点：

#### 只支持等值join

In [ ]:
eg :
    SELECT a.* FROM a JOIN b ON (a.id = b.id)
    SELECT a.* FROM a JOIN b
    ON (a.id = b.id AND a.department = b.department)

#### 可以join 多于2个表

In [ ]:
eg:
     SELECT a.val, b.val, c.val FROM a JOIN b
    ON (a.key = b.key1) JOIN c ON (c.key = b.key2)

In [ ]:
 eg:
    SELECT a.val, b.val, c.val FROM a JOIN b
    ON (a.key = b.key1) JOIN c
    ON (c.key = b.key1)

In [ ]:
eg:
    
SELECT a.val, b.val, c.val FROM a JOIN b ON (a.key = b.key1)
  JOIN c ON (c.key = b.key2)

    #join 被转化为 2 个 map/reduce 任务。因为 b.key1 用于第一次 join 条件，而 b.key2 用于第二次 join。

#### join 时，每次 map/reduce 任务的逻辑

In [ ]:
eg:
     SELECT a.val, b.val, c.val FROM a
    JOIN b ON (a.key = b.key1) JOIN c ON (c.key = b.key1)
'''
所有表都使用同一个 join key（使用 1 次 map/reduce 任务计算）。
Reduce 端会缓存 a 表和 b 表的记录，然后每次取得一个 c 表的记录就计算一次 join 结果
'''

In [ ]:
eg:
     SELECT a.val, b.val, c.val FROM a
    JOIN b ON (a.key = b.key1) JOIN c ON (c.key = b.key2)
'''
这里用了 2 次 map/reduce 任务。第一次缓存 a 表，用 b 表序列化；
第二次缓存第一次 map/reduce 任务的结果，然后用 c 表序列化。
'''

#### LEFT，RIGHT 和 FULL OUTER 关键字用于处理 join 中空记录的情况

In [ ]:
eg:
    SELECT a.val, b.val FROM a LEFT OUTER
    JOIN b ON (a.key=b.key)

In [ ]:
eg:
      SELECT a.val, b.val FROM a
  LEFT OUTER JOIN b ON (a.key=b.key)
  WHERE a.ds='2009-07-07' AND b.ds='2009-07-07'

In [ ]:
  eg:
    SELECT a.val, b.val FROM a LEFT OUTER JOIN b
  ON (a.key=b.key AND
      b.ds='2009-07-07' AND
      a.ds='2009-07-07')
'''
这一查询的结果是预先在 join 阶段过滤过的，所以不会存在上述问题。这一逻辑也可以应用于 RIGHT 和 FULL 类型的 join 中。
'''

In [ ]:
eg:
     SELECT a.val1, a.val2, b.val, c.val
  FROM a
  JOIN b ON (a.key = b.key)
  LEFT OUTER JOIN c ON (a.key = c.key)
'''
join a 表到 b 表，丢弃掉所有 join key 中不匹配的记录，然后用这一中间结果和 c 表做 join。
这一表述有一个不太明显的问题，就是当一个 key 在 a 表和 c 表都存在，但是 b 表中不存在的时候：
整个记录在第一次 join，即 a JOIN b 的时候都被丢掉了（包括a.val1，a.val2和a.key），
然后我们再和 c 表 join 的时候，如果 c.key 与 a.key 或 b.key 相等，就会得到这样的结果：
NULL, NULL, NULL, c.val。（不会这样）
'''

#### LEFT SEMI JOIN 是 IN/EXISTS 子查询的一种更高效的实现。

In [ ]:
eg:
  SELECT a.key, a.value
  FROM a
  WHERE a.key in
   (SELECT b.key
    FROM B);
可以被重写为：
   SELECT a.key, a.val
   FROM a LEFT SEMI JOIN b on (a.key = b.key)

# HSQL中join与left-join区别

## Inner Join

In [ ]:
presto:default> select count(*)
             -> from dw.kn2_ord_order_detail_daily_change t1
             ->  join dw.kn2_ehr_person_organization t2 on t1.follow_saler_id=t2.empl_id  and  t2.dt='20180813'
             -> where  t1.dt='20180813';

![title](img/innerjoin1.png)

In [ ]:
公共部分的数据才会被查询出来；

![title](img/innerjoin.png)

## Left Join

In [ ]:
presto:default> select count(*)
             -> from dw.kn2_ord_order_detail_daily_change t1
             ->  left join dw.kn2_ehr_person_organization t2 on t1.follow_saler_id=t2.empl_id  and  t2.dt='20180813'
             -> where  t1.dt='20180813';

![title](img/leftjoin1.png)

![title](img/leftjoin.png)

# left join on and 与 left join on where的区别

- **on条件是在生成临时表时使用的条件，它不管on中的条件是否为真，都会返回左边表中的记录**
- **where条件是在临时表生成好后，再对临时表进行过滤的条件。这时已经没有left join的含义(必须返回左边表的记录)，条件不为真的就全部过滤掉。**

## 示例

| id   | size     |
|-----|------------
| 1    | 10  |
| 2    | 20  |
| 3    | 30   |
| 4    | 40   |

In [ ]:
产生表语句:
    create table tab1 as 
    select 1 as id1,10 as size from dual
    union all
    select 2 as id1,20 as size from dual
    union all 
    select 3 as id1,30 as size from dual
    union all
    select 4 as id1,40 as size from dual;

| size   | name     |
|-----|------------
| 10    | AAA  |
| 20    | BBB  |
| 30   | CCC   |
| 30   | DDD   |
| 50   | EEE   |

In [ ]:
产生表语句:
    create table tab2 as 
    select 10 as size,'AAA' as name from dual
    union all
    select 20 as size,'BBB' as name from dual
    union all 
    select 30 as size,'CCC' as name from dual
    union all
    select 30 as size,'DDD' as name from dual
    union all 
    select 50 as size,'EEE' as name from dual;

In [ ]:
两条SQL：
    select * from tab1 left join tab2 on (tab1.size = tab2.size) where tab2.name='AAA';
    select * from tab1 left join tab2 on (tab1.size = tab2.size and tab2.name='AAA');

**先看left join on where 选择结果 首先它会关联生成临时表数据;然后在通过where条件进行筛选**

![title](img/join_on_where.png)

**接下来我们在分析left join on and处理过程，它的处理过程是先将右侧表以and为条件进行筛选 再和左侧表进行关联查询；最终的结果就是以左侧表为基础 将右侧符合条件的数据置为null 结果如下; **

![title](img/join_on_and.png)

### left join 

In [ ]:
presto:default> select t1.id1 ,t1.size,t2.name from tab1 t1 left join tab2 t2 on t1.size=t2.size;
 id1 | size | name 
-----+------+------
   4 |   40 | NULL 
   3 |   30 | DDD  
   3 |   30 | CCC  
   1 |   10 | AAA  
   2 |   20 | BBB  
(5 rows)

### right join

In [ ]:
presto:default> select t1.id1 ,t1.size,t2.name from tab1 t1 right join tab2 t2 on t1.size=t2.size;
 id1  | size | name 
------+------+------
    2 |   20 | BBB  
    1 |   10 | AAA  
 NULL | NULL | EEE  
    3 |   30 | DDD  
    3 |   30 | CCC  
(5 rows)

### join

In [ ]:
presto:default> select t1.id1 ,t1.size,t2.name from tab1 t1 join tab2 t2 on t1.size=t2.size;
 id1 | size | name 
-----+------+------
   2 |   20 | BBB  
   3 |   30 | DDD  
   3 |   30 | CCC  
   1 |   10 | AAA  
(4 rows)

### full join

In [ ]:
presto:default> select t1.id1 ,t1.size,t2.name from tab1 t1 full join tab2 t2 on t1.size=t2.size;
 id1  | size | name 
------+------+------
    3 |   30 | DDD  
    3 |   30 | CCC  
    1 |   10 | AAA  
    4 |   40 | NULL 
 NULL | NULL | EEE  
    2 |   20 | BBB  
(6 rows)

In [ ]:
presto:default> select *from tab1;
 id1 | size 
-----+------
   1 |   10 
   2 |   20 
   3 |   30 
   4 |   40 
(4 rows)

presto:default> select *from tab2;
 size | name 
------+------
   10 | AAA  
   20 | BBB  
   30 | CCC  
   30 | DDD  
   50 | EEE  
(5 rows)

presto:default> select *from tab1 t1 full join tab2 t2 on t1.size=t2.size;
 id1  | size | size | name 
------+------+------+------
    1 |   10 |   10 | AAA  
    2 |   20 |   20 | BBB  
 NULL | NULL |   50 | EEE  
    3 |   30 |   30 | DDD  
    3 |   30 |   30 | CCC  
    4 |   40 | NULL | NULL 
(6 rows)

hive (default)> select id1,nvl(t1.size,t2.size) as size,name from tab1 t1 full join tab2 t2 on t1.size=t2.size;
OK
id1	size	name
1	10	AAA
2	20	BBB
3	30	DDD
3	30	CCC
4	40	NULL
NULL	50	EEE



## 检查 逻辑 或 的用法

In [ ]:
产生表语句：
    create table tab3 as
    select t1.id1 ,t1.size,t2.name from tab1 t1 full join tab2 t2 on t1.size=t2.size

| id1  | size | name| 
|------|------|------|
 |   3 |   30 | DDD  |
 |   3 |   30 | CCC  |
 |   1 |   10 | AAA  |
 |   4 |   40 | NULL |
 |NULL | NULL | EEE  |
|    2 |   20 | BBB |

In [ ]:
    create table tab4 as
    select id1,size,name from tab3
    union all 
    select  99 as id1,null as size,'TTT' as name from dual
    union all 
    select  8 as id1,null as size,'HHH' as name from dual
    union all 
    select  null as id1,76 as size,'HHH' as name from dual
    ;

| id1  | size | name| 
|-----|-----|-----|
 |   1 |   10 | AAA  |
 |   2 |   20 | BBB  |
 |   3 |   30 | DDD  |
 |   3 |   30 | CCC  |
 |   4 |   40 | NULL |
| NULL | NULL | EEE  |
|   99 | NULL | TTT  |
|    8 | NULL | HHH  |
| NULL |   76 | HHH|

In [ ]:
presto:default> select *from tab4 where id1 is not null or size is not null;
 id1  | size | name 
------+------+------
    1 |   10 | AAA  
    2 |   20 | BBB  
    3 |   30 | DDD  
    3 |   30 | CCC  
    4 |   40 | NULL 
   99 | NULL | TTT  
    8 | NULL | HHH  
 NULL |   76 | HHH  
(8 rows)

## Union all 与Union 的区别

### 操作 tab3 和 tab4 Union all

![title](img/UnionAll.png)

### 操作 tab3 和 tab4 Union 

操作语句： 
    select *from tab3 union all select *from tab4;

![title](img/Union.png)